In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

Load API key from .env file

In [2]:
KEY = os.environ.get("GEMINI_API_KEY")

Gemini

In [3]:
from google import genai
from google.genai import types

In [4]:
# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client(api_key=KEY)  # here you can also pass the api_key directly using os.environ['GEMINI_API_KEY']

default_model = "gemini-2.5-flash"

In [5]:
response = client.models.generate_content(
    model=default_model,
    contents="Explain cat photos",
    config=types.GenerateContentConfig(
        max_output_tokens=50,
        thinking_config=types.ThinkingConfig(thinking_budget=0) # Disables thinking
    ),
)
print(response.text)

Cat photos are a pervasive and much-loved internet phenomenon that derive their appeal from a fascinating blend of human psychology, feline characteristics, and digital culture. They're not just pictures of cats; they're a complex and often humorous form of communication


RAG

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings  # get embeddings from Gemini
from langchain_community.vectorstores import FAISS  # "db" to store and retrieve embeddings
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter  # split long documents
from pdfminer.high_level import extract_text  # extract text from pdfs
from langchain_huggingface import HuggingFaceEmbeddings

In [7]:
from pathlib import Path

In [8]:
#DOC_PATH = "./scraped_pdfs/32018R0213.pdf"
dir_path = Path("./scraped_pdfs")

In [ ]:
import logging
# Silence pdfminer logs below ERROR
logging.getLogger("pdfminer").setLevel(logging.ERROR)
text_lst = []
# Extract text
#text = extract_text(DOC_PATH, page_numbers=[0])
i = 0
for file in dir_path.iterdir():
    if i >= 20:
        break
    if file.name.endswith(".pdf") and not file.name.startswith("slvfs"):
        text = extract_text(file)
        text_lst.append(text)
    else:
        print(f"Skipping {file.name}")
    i += 1

1.2.2002

SV

Europeiska gemenskapernas officiella tidning

L 31/1

I

(Rättsakter vilkas publicering är obligatorisk)

EUROPAPARLAMENTETS OCH RÅDETS FÖRORDNING (EG) nr 178/2002

av den 28 januari 2002

om allmänna principer och krav för livsmedelslagstiftning, om inrättande av Europeiska myndig-
heten för livsmedelssäkerhet och om förfaranden i frågor som gäller livsmedelssäkerhet

EUROPAPARLAMENTET OCH EUROPEISKA UNIONENS RÅD HAR
ANTAGIT DENNA FÖRORDNING

med beaktande av Fördraget om upprättandet av Europeiska
gemenskapen, särskilt artiklarna 37, 95, 133 och 152.4 b i
detta,

med beaktande av kommissionens förslag (1),

med beaktande av Ekonomiska och sociala kommitténs
yttrande (2),

med beaktande av Regionkommitténs yttrande (3),

i enlighet med förfarandet i artikel 251 i fördraget (4), och

av följande skäl:

(1)

(2)

(3)

Den fria rörligheten för säkra och hälsosamma livsmedel
är en nödvändig del av den inre marknaden och den
bidrar i hög grad till att skydda medborgarnas häls

In [15]:
print(len(text_lst))

18


In [10]:
# Wrap as a Document
doc = Document(page_content=text, metadata={"source": "32018R0213.pdf", "page": 1})

In [11]:
# 2. Split the documents into small chunks

splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
docs = splitter.split_documents([doc])

for i, d in enumerate(docs, 1):
   print(f"Chunk {i}: {d.page_content[:60]}... | Metadata: {d.metadata}")

Chunk 1: 23.9.2021 

SV

Europeiska  unionens  officiella  tidning 

... | Metadata: {'source': '32018R0213.pdf', 'page': 1}
Chunk 2: om komplettering av Europaparlamentets och rådets förordning... | Metadata: {'source': '32018R0213.pdf', 'page': 1}
Chunk 3: för  erkännande  av kontrollmyndigheter  och  kontrollorgan ... | Metadata: {'source': '32018R0213.pdf', 'page': 1}
Chunk 4: ekologiskt  certifierade  aktörer  och  aktörsgrupper  och  ... | Metadata: {'source': '32018R0213.pdf', 'page': 1}
Chunk 5: med  regler  om  deras  tillsyn  och  de  kontroller  och  a... | Metadata: {'source': '32018R0213.pdf', 'page': 1}
Chunk 6: (Text av betydelse för EES) 

EUROPEISKA KOMMISSIONEN HAR AN... | Metadata: {'source': '32018R0213.pdf', 'page': 1}
Chunk 7: med  beaktande  av  Europaparlamentets  och  rådets  förordn... | Metadata: {'source': '32018R0213.pdf', 'page': 1}
Chunk 8: produktion och märkning av ekologiska produkter och om upphä... | Metadata: {'source': '32018R0213.pdf', 'page': 1}


In [12]:
# Local embedding model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
# Database
db = FAISS.from_documents(docs, embeddings)

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

In [ ]:
# 5. Given a query, retrieve the most relevant chunk(s) and appropriately prompt your LLM

query = "Bisfenol A"
results = db.similarity_search(query, k=10)

for r in results:
    print(r.page_content)

GUI

In [ ]:
import gradio as gr